In [1]:
import geopandas as gpd
from  imgw_static import get_map_zlewnie, get_static_data_hydro
from imgw_api import get_hydro_metadata, get_meteo_metadata
import pandas as pd
import numpy as np

# Preprocessing

## Dane Hydrologiczne

In [2]:
import pandas as pd

df = get_static_data_hydro()

Dane Pomiarowe

In [3]:
print("Shape: ", df.shape)
print("Ilość stacji hydro w danych pomiarowych: ",df["Station Code"].nunique())
df.head()


Shape:  (4273340, 10)
Ilość stacji hydro w danych pomiarowych:  917


,Station Code,Station Name,Name,Hydro Year,Hydro Month,Day,Water Level,Flow,Water Temp,Calendar Month
0,149180020,CHA£UPKI,Odra (1),2010,1,1,158,30.8,99.9,11
1,149180020,CHA£UPKI,Odra (1),2010,1,2,154,28.5,99.9,11
2,149180020,CHA£UPKI,Odra (1),2010,1,3,150,26.0,99.9,11
3,149180020,CHA£UPKI,Odra (1),2010,1,4,149,25.4,99.9,11
4,149180020,CHA£UPKI,Odra (1),2010,1,5,149,25.9,99.9,11


Dane Meta

In [4]:
gdf = get_hydro_metadata()
print("Shape: ", gdf.shape)
print("Ilość stacji hydro w meta danych: ",gdf["Station Code"].nunique())
gdf.head()

Shape:  (859, 5)
Ilość stacji hydro w meta danych:  859


,Station Code,Station Name,Lon,Lat,geometry
0,150160330,SZCZYTNA,16.443056,50.415556,POINT (318401.478 286283.506)
1,150160340,SARNY,16.465833,50.547778,POINT (320520.661 300923.221)
2,150160350,SZALEJÓW GÓRNY,16.537222,50.418333,POINT (325098.752 286366.241)
3,150160360,STARKÓW,16.580000,50.377500,POINT (327988.637 281728.229)
4,150160370,TOPOLICE,16.609167,50.366944,POINT (330023.848 280487.958)


## Wstępna selekcja danych hydrologicznych

Wybieramy stacje które:

1. Rozpoczęły pomiary przed lub w 2010 roku.
2. Można zmapować do ich lokalizacji.

In [5]:
df['Calendar Year'] = np.where(df['Hydro Month'] <3, df['Hydro Year']-1, df['Hydro Year'])
df = df.loc[df['Calendar Year']!=2023, :]
df

,Station Code,Station Name,Name,Hydro Year,Hydro Month,Day,Water Level,Flow,Water Temp,Calendar Month,Calendar Year
0,149180020,CHA£UPKI,Odra (1),2010,1,1,158,30.80,99.9,11,2009
1,149180020,CHA£UPKI,Odra (1),2010,1,2,154,28.50,99.9,11,2009
2,149180020,CHA£UPKI,Odra (1),2010,1,3,150,26.00,99.9,11,2009
3,149180020,CHA£UPKI,Odra (1),2010,1,4,149,25.40,99.9,11,2009
4,149180020,CHA£UPKI,Odra (1),2010,1,5,149,25.90,99.9,11,2009
...,...,...,...,...,...,...,...,...,...,...,...
301685,149190250,JABŁONKA,Piekielnik (82224),2023,2,27,161,3.08,99.9,12,2022
301686,149190250,JABŁONKA,Piekielnik (82224),2023,2,28,157,2.42,99.9,12,2022
301687,149190250,JABŁONKA,Piekielnik (82224),2023,2,29,153,1.80,99.9,12,2022
301688,149190250,JABŁONKA,Piekielnik (82224),2023,2,30,151,1.32,99.9,12,2022


Odcinamy stacje z pierszwego warunku.

In [6]:
valid_stations = df.groupby("Station Code")["Calendar Year"].min()
valid_stations = valid_stations[valid_stations <= 2010]

# Step 2: Filter the DataFrame to include only those stations
filtered_df = df[df["Station Code"].isin(valid_stations.index)]

In [7]:
filtered_df['Station Code'].nunique()

874

Odcinamy stacje z drugiego warunku.

In [8]:
valid_stations = gdf['Station Code']

# Step 3: Filter the DataFrame to include only those stations
filtered_df_2 = filtered_df[filtered_df["Station Code"].isin(valid_stations)]

In [9]:
print("Shape: ", filtered_df_2.shape)
print("Ilość stacji w odfiltrowanych hydro danych: ",filtered_df_2["Station Code"].nunique())

Shape:  (3419100, 11)
Ilość stacji w odfiltrowanych hydro danych:  719


### Mapowanie Kodów 

Stan wody 9999 oznacza brak danych w bazie.

Przepływ 99999.999 oznacza, że przepływ w tym dniu nie był opracowywany.

Temperatura 99.9 oznacza brak danych w bazie, która może wynikać np. z braku pomiarów temperatury na stacji.

In [10]:
import numpy as np

filtered_df_2.replace({
    'Water Level': {9999: np.nan},
    'Flow': {99999.999: np.nan},
    'Water Temp': {99.9: np.nan}
}, inplace=True)

C:\Users\Tymek\AppData\Local\Temp\ipykernel_25928\3020362930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2.replace({


Eksportujemy odfiltrowane dane do static_data

W efekcie mamy spójne dane zaczynające się od 2010 i z lokalizacją i z zmapowanymi wartościami brakującymi.

In [11]:
filtered_df_2.to_parquet("../static_data/filtered_hydro_data.parquet.gzip")

## Dane Opad

In [12]:
df = pd.read_parquet('../static_data/meteo_opad_data.parquet.gzip')

In [13]:
print("Shape: ", df.shape)
print("Ilość stacji meteo opadowych w danych pomiarowych: ",df["Station Code"].nunique())

Shape:  (1801634, 16)
Ilość stacji meteo opadowych w danych pomiarowych:  1137


In [14]:
gdf = get_meteo_metadata()
print("Shape: ", gdf.shape)
print("Ilość stacji hydro w meta danych: ",gdf["Station Code"].nunique())

Shape:  (766, 5)
Ilość stacji hydro w meta danych:  766


## Wstępna selekcja danych meteorologicznych

Wybieramy stacje które:

1. Rozpoczęły pomiary przed lub w 2010 roku.
2. Można zmapować do ich lokalizacji.

In [15]:
valid_stations = df.groupby("Station Code")["Year"].min()
valid_stations = valid_stations[valid_stations <= 2010].index

# Step 2: Filter the DataFrame to include only those stations
filtered_df = df[df["Station Code"].isin(valid_stations)]

In [16]:
print("Ilość stacji w odfiltrowanych (filtracja czasowa) meteo danych: ",filtered_df["Station Code"].nunique())

Ilość stacji w odfiltrowanych (filtracja czasowa) meteo danych:  958


In [17]:
valid_stations = gdf['Station Code']

# Step 3: Filter the DataFrame to include only those stations
filtered_df_2 = filtered_df[filtered_df["Station Code"].isin(valid_stations)]

In [18]:
print("Shape: ", filtered_df_2.shape)
print("Ilość stacji w odfiltrowanych meteo danych: ",filtered_df_2["Station Code"].nunique())

Shape:  (605780, 16)
Ilość stacji w odfiltrowanych meteo danych:  274


In [19]:
date_str = filtered_df_2['Year'].astype(str) + '-' + \
                        filtered_df_2['Month'].astype(str).str.zfill(2) + '-' + \
                        filtered_df_2['Day'].astype(str).str.zfill(2)
filtered_df_2['Calendar Date'] = pd.to_datetime(date_str, errors='coerce')

C:\Users\Tymek\AppData\Local\Temp\ipykernel_25928\3497555919.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2['Calendar Date'] = pd.to_datetime(date_str, errors='coerce')


In [20]:
filtered_df_2

,Station Code,Station Name,Year,Month,Day,Daily Precip Sum,SMDB Measurement Status,Precipitation Type,Snow Cover Height,PKSN Measurement Status,Fresh Snow Height,HSS Measurement Status,Snow Type,GATS Measurement Status,Snow Cover Type,RPSN Measurement Status,Calendar Date
0,249180020,WARSZOWICE,2010,1,1,1.1,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-01
1,249180020,WARSZOWICE,2010,1,2,5.1,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-02
2,249180020,WARSZOWICE,2010,1,3,0.6,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-03
3,249180020,WARSZOWICE,2010,1,4,0.3,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-04
4,249180020,WARSZOWICE,2010,1,5,3.3,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13722,254220060,BANIE MAZURSKIE,2022,12,27,0.2,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN,2022-12-27
13723,254220060,BANIE MAZURSKIE,2022,12,28,5.1,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN,2022-12-28
13724,254220060,BANIE MAZURSKIE,2022,12,29,1.9,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN,2022-12-29
13725,254220060,BANIE MAZURSKIE,2022,12,30,0.3,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN,2022-12-30


In [21]:
filtered_df_2.to_parquet("../static_data/filtered_meteo_opad_data.parquet.gzip")